<a href="https://colab.research.google.com/github/Nivea5678/Yeti-Analytics/blob/main/Optim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import opinion_lexicon
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU
from nltk.corpus import stopwords
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.layers import Bidirectional, Dense, Dropout, Embedding
from tensorflow.keras.optimizers import Adam

In [ ]:
# loading the data
from google.colab import files
uploaded = files.upload()

Saving Yeti Analytics.xlsx to Yeti Analytics.xlsx


In [ ]:
# Step 1: Load the dataset into dataframe
df = pd.read_excel('Yeti Analytics.xlsx')
df

,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5
...,...,...
20486,"best kept secret 3rd time staying charm, not 5...",5
20487,great location price view hotel great quick pl...,4
20488,"ok just looks nice modern outside, desk staff ...",2
20489,hotel theft ruined vacation hotel opened sept ...,1


In [ ]:
# Download NLTK resources
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Preprocessing the text
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha()]
    tokens = [token for token in tokens if token not in stop_words]
    return ' '.join(tokens)

df['Review'] = df['Review'].apply(preprocess_text)

In [ ]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import nltk
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = text.lower()  # Convert to lowercase
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]  # Lemmatization and stopword removal
    return ' '.join(words)

df['Cleaned_Review'] = df['Review'].apply(preprocess_text)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
!wget -P "/content/gdrive/My Drive/glove_data" "http://nlp.stanford.edu/data/glove.6B.zip"
!unzip "/content/gdrive/My Drive/glove_data/glove.6B.zip" -d "/content/gdrive/My Drive/glove_data/"

--2023-08-04 20:21:26--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-08-04 20:21:26--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-08-04 20:21:26--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘/content/gdrive/My 

In [ ]:
# Convert all non-positive ratings to negative
df['Rating'] = df['Rating'].apply(lambda x: 'positive' if x == 'positive' else 'negative')

# Splitting into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df['Cleaned_Review'], df['Rating'], test_size=0.7, random_state=42)

# Tokenization and Padding
max_sequence_length = 100
vocab_size = 10000

tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(X_train)

X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_sequence_length)

# Mapping class labels to binary values
y_train_binary = (y_train == 'positive').astype(int)
y_test_binary = (y_test == 'positive').astype(int)

# Mapping class labels to start from 0
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# One-hot encoding the labels
onehot_encoder = OneHotEncoder(sparse=False)
y_train_onehot = onehot_encoder.fit_transform(y_train_encoded.reshape(-1, 1))
y_test_onehot = onehot_encoder.transform(y_test_encoded.reshape(-1, 1))

# Modify the vocab_size if needed
vocab_size = len(tokenizer.word_index) + 1  # Add 1 for the unknown word token

embedding_dim = 100
embedding_index = {}
with open("/content/gdrive/My Drive/glove_data/glove.6B.100d.txt", encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs

embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Update the model with pre-trained embeddings
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_sequence_length, weights=[embedding_matrix], trainable=False))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Bidirectional(LSTM(32)))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  # Binary classification with sigmoid activation

# Rest of the code remains unchanged

# ... (continue with the rest of the code)
# Compiling and train the model with custom learning rate
optimizer = Adam(learning_rate=0.001)  # Experiment with different learning rates
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.fit(X_train_padded, y_train_binary, validation_data=(X_test_padded, y_test_binary), epochs=5, batch_size=32)

# Evaluating the model
_, accuracy = model.evaluate(X_test_padded, y_test_binary)
print('Accuracy: %.2f%%' % (accuracy * 100))



/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Epoch 1/5
193/193 [==============================] - 83s 365ms/step - loss: 0.0294 - accuracy: 0.9924 - val_loss: 8.8094e-06 - val_accuracy: 1.0000
Epoch 2/5
193/193 [==============================] - 85s 440ms/step - loss: 2.7561e-04 - accuracy: 1.0000 - val_loss: 1.2260e-06 - val_accuracy: 1.0000
Epoch 3/5
193/193 [==============================] - 83s 433ms/step - loss: 1.2262e-04 - accuracy: 1.0000 - val_loss: 3.1944e-07 - val_accuracy: 1.0000
Epoch 4/5
193/193 [==============================] - 83s 429ms/step - loss: 7.6581e-05 - accuracy: 1.0000 - val_loss: 1.2980e-07 - val_accuracy: 1.0000
Epoch 5/5
449/449 [==============================] - 29s 64ms/step - loss: 5.2615e-08 - accuracy: 1.0000
Accuracy: 100.00%


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Bidirectional, Dense, Dropout, Embedding, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np

# ... (previous code up to line 36)

# Convert all non-positive ratings to negative
df['Rating'] = df['Rating'].apply(lambda x: 'positive' if x == 'positive' else 'negative')

# Splitting into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df['Cleaned_Review'], df['Rating'], test_size=0.2, random_state=42)  # Change test_size to 0.2

# Tokenization and Padding
max_sequence_length = 100
vocab_size = 10000

tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(X_train)

X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_sequence_length)

# Mapping class labels to binary values
y_train_binary = (y_train == 'positive').astype(int)
y_test_binary = (y_test == 'positive').astype(int)

# Modify the vocab_size if needed
vocab_size = len(tokenizer.word_index) + 1  # Add 1 for the unknown word token

embedding_dim = 100
embedding_index = {}
with open("/content/gdrive/My Drive/glove_data/glove.6B.100d.txt", encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs

embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Update the model with pre-trained embeddings
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_sequence_length, weights=[embedding_matrix], trainable=False))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Dropout(0.5))  # Add dropout after LSTM
model.add(Bidirectional(LSTM(32)))
model.add(Dropout(0.5))  # Add dropout after LSTM
model.add(Dense(16, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))  # Add dropout after dense layer
model.add(Dense(1, activation='sigmoid'))  # Binary classification with sigmoid activation

# Compiling the model
optimizer = Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Early Stopping Callback
early_stopping = EarlyStopping(patience=3, restore_best_weights=True)

# Training the model
model.fit(X_train_padded, y_train_binary, validation_data=(X_test_padded, y_test_binary), epochs=5, batch_size=32, callbacks=[early_stopping])

# Evaluating the model
_, accuracy = model.evaluate(X_test_padded, y_test_binary)
print('Accuracy: %.2f%%' % (accuracy * 100))


Epoch 1/5
513/513 [==============================] - 144s 264ms/step - loss: 0.0217 - accuracy: 0.9941 - val_loss: 1.2873e-08 - val_accuracy: 1.0000
Epoch 2/5
513/513 [==============================] - 131s 255ms/step - loss: 2.0758e-04 - accuracy: 1.0000 - val_loss: 1.7374e-10 - val_accuracy: 1.0000
Epoch 3/5
513/513 [==============================] - 129s 252ms/step - loss: 7.0688e-05 - accuracy: 1.0000 - val_loss: 1.3167e-11 - val_accuracy: 1.0000
Epoch 4/5
513/513 [==============================] - 127s 247ms/step - loss: 8.7841e-05 - accuracy: 1.0000 - val_loss: 1.1566e-12 - val_accuracy: 1.0000
Epoch 5/5
129/129 [==============================] - 9s 69ms/step - loss: 1.7073e-13 - accuracy: 1.0000
Accuracy: 100.00%


In [ ]:


# Update the model with pre-trained embeddings
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_sequence_length, weights=[embedding_matrix], trainable=False))
model.add(Bidirectional(LSTM(32, return_sequences=True)))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(16)))
model.add(Dropout(0.5))
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compiling the model
optimizer = Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Early Stopping Callback
early_stopping = EarlyStopping(patience=5, restore_best_weights=True)

# Training the model
model.fit(X_train_padded, y_train_binary, validation_data=(X_test_padded, y_test_binary), epochs=10, batch_size=64, callbacks=[early_stopping])

# Evaluating the model
_, accuracy = model.evaluate(X_test_padded, y_test_binary)
print('Accuracy: %.2f%%' % (accuracy * 100))


Epoch 1/20
257/257 [==============================] - 84s 250ms/step - loss: 0.0601 - accuracy: 0.9942 - val_loss: 9.7421e-07 - val_accuracy: 1.0000
Epoch 2/20
257/257 [==============================] - 58s 224ms/step - loss: 0.0229 - accuracy: 0.9996 - val_loss: 2.8528e-08 - val_accuracy: 1.0000
Epoch 3/20
257/257 [==============================] - 60s 232ms/step - loss: 0.0195 - accuracy: 0.9998 - val_loss: 3.0764e-09 - val_accuracy: 1.0000
Epoch 4/20
257/257 [==============================] - 57s 220ms/step - loss: 0.0179 - accuracy: 0.9999 - val_loss: 6.2612e-10 - val_accuracy: 1.0000
Epoch 5/20
257/257 [==============================] - 55s 213ms/step - loss: 0.0180 - accuracy: 1.0000 - val_loss: 1.6013e-10 - val_accuracy: 1.0000
Epoch 6/20
257/257 [==============================] - 57s 222ms/step - loss: 0.0152 - accuracy: 1.0000 - val_loss: 5.4202e-11 - val_accuracy: 1.0000
Epoch 7/20
257/257 [==============================] - 55s 216ms/step - loss: 0.0128 - accuracy: 1.0000 - v

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Bidirectional, Dense, Dropout, Embedding, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# ... (previous code up to line 36)

# Convert all non-positive ratings to negative
df['Rating'] = df['Rating'].apply(lambda x: 'positive' if x == 'positive' else 'negative')

# Using a smaller subset of the data
subset_size = 10000  # Choose a smaller subset size
df_subset = df.sample(n=subset_size, random_state=42)

# Splitting into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df_subset['Cleaned_Review'], df_subset['Rating'], test_size=0.2, random_state=42)

# Tokenization and Padding
max_sequence_length = 100
vocab_size = 10000

tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(X_train)

X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_sequence_length)

# Mapping class labels to binary values
label_encoder = LabelEncoder()
y_train_binary = label_encoder.fit_transform(y_train)
y_test_binary = label_encoder.transform(y_test)

# Modify the vocab_size if needed
vocab_size = len(tokenizer.word_index) + 1  # Add 1 for the unknown word token

embedding_dim = 100
embedding_index = {}
with open("/content/gdrive/My Drive/glove_data/glove.6B.100d.txt", encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs

embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Update the model with pre-trained embeddings
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_sequence_length, weights=[embedding_matrix], trainable=False))
model.add(Bidirectional(LSTM(32, return_sequences=True)))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(16)))
model.add(Dropout(0.5))
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compiling the model
optimizer = Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Early Stopping Callback
early_stopping = EarlyStopping(patience=5, restore_best_weights=True)

# Training the model
model.fit(X_train_padded, y_train_binary, validation_data=(X_test_padded, y_test_binary), epochs=10, batch_size=64, callbacks=[early_stopping])

# Evaluating the model
_, accuracy = model.evaluate(X_test_padded, y_test_binary)
print('Accuracy: %.2f%%' % (accuracy * 100))


Epoch 1/10
125/125 [==============================] - 48s 251ms/step - loss: 0.1843 - accuracy: 0.9744 - val_loss: 0.0011 - val_accuracy: 1.0000
Epoch 2/10
125/125 [==============================] - 28s 226ms/step - loss: 0.0506 - accuracy: 0.9966 - val_loss: 9.9218e-05 - val_accuracy: 1.0000
Epoch 3/10
125/125 [==============================] - 27s 216ms/step - loss: 0.0445 - accuracy: 0.9985 - val_loss: 2.1084e-05 - val_accuracy: 1.0000
Epoch 4/10
125/125 [==============================] - 26s 211ms/step - loss: 0.0422 - accuracy: 0.9994 - val_loss: 6.9187e-06 - val_accuracy: 1.0000
Epoch 5/10
125/125 [==============================] - 26s 206ms/step - loss: 0.0415 - accuracy: 0.9996 - val_loss: 2.5205e-06 - val_accuracy: 1.0000
Epoch 6/10
125/125 [==============================] - 25s 201ms/step - loss: 0.0347 - accuracy: 1.0000 - val_loss: 1.1162e-06 - val_accuracy: 1.0000
Epoch 7/10
125/125 [==============================] - 27s 216ms/step - loss: 0.0362 - accuracy: 0.9999 - val_l

In [ ]:


# Update the model with pre-trained embeddings
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_sequence_length, weights=[embedding_matrix], trainable=False))
model.add(Bidirectional(LSTM(32, return_sequences=True, recurrent_dropout=0.2)))  # Apply recurrent dropout
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(16, recurrent_dropout=0.2)))  # Apply recurrent dropout
model.add(Dropout(0.5))
model.add(Dense(8, activation='relu', kernel_regularizer='l2'))  # Apply L2 regularization
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid', kernel_regularizer='l2'))  # Apply L2 regularization

# Compiling the model
optimizer = Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Early Stopping and Learning Rate Scheduler Callbacks
early_stopping = EarlyStopping(patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(factor=0.5, patience=2, verbose=1)

# Training the model
model.fit(X_train_padded, y_train_binary, validation_data=(X_test_padded, y_test_binary), epochs=30, batch_size=64, callbacks=[early_stopping, reduce_lr])

# Evaluating the model
_, accuracy = model.evaluate(X_test_padded, y_test_binary)
print('Accuracy: %.2f%%' % (accuracy * 100))


Epoch 1/30
125/125 [==============================] - 69s 461ms/step - loss: 0.2747 - accuracy: 0.9494 - val_loss: 0.0859 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 2/30
125/125 [==============================] - 54s 436ms/step - loss: 0.1549 - accuracy: 0.9910 - val_loss: 0.0518 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 3/30
125/125 [==============================] - 58s 461ms/step - loss: 0.1248 - accuracy: 0.9995 - val_loss: 0.0357 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 4/30
125/125 [==============================] - 55s 442ms/step - loss: 0.1094 - accuracy: 1.0000 - val_loss: 0.0283 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 5/30
125/125 [==============================] - 54s 434ms/step - loss: 0.1002 - accuracy: 1.0000 - val_loss: 0.0250 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 6/30
125/125 [==============================] - 59s 471ms/step - loss: 0.0932 - accuracy: 1.0000 - val_loss: 0.0233 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 7/30
125/125 [==============================] 